In [1]:
import open3d as o3d
import numpy as np
import pyrealsense2 as rs
import cv2
import os
import sys
import shutil
import json

## Directory, path setting

In [2]:
from utils.perception_config import *
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/perception'))

cfg_off = '--src_path='
color_off = '--color_path='
depth_off = '--depth_path='
img_num_off = '--img_num='
output_off = '--output_path='

In [3]:
from scripts.perception.utils.function_utils import *

## Acquire RGBD images

In [5]:
streaming(DATASET_DIR)

/home/jhkim/Projects/rnb-planning/src/scripts/perception/dataset not empty. Overwrite? (y/n) : n


## Scene Reconstruction
#### Issue : Not implemented yet. Further development is needed.

In [6]:
from boost_reconstruction.reconstruction_interface import reconstruction_interface_py as rci

In [7]:
rci.getReconstruction(COLOR_PATH, DEPTH_PATH, INTRINSIC_PATH, WORKING_DIR)

'========= Complete 3D Reconstruction ========='

In [8]:
# Construct Camera_parameter Class
# It has intrinsic, extrinsic parameter
cam_params = Camera_parameter(INTRINSIC_PATH, EXTRINSIC_PATH)

In [9]:
img_num = len(cam_params.cam_traj)

In [10]:
# Check img_num
img_num

173

## Segmentation 
#### Issue : Detection result is sometimes weird

In [11]:
import subprocess

In [12]:
cfg_path_arg = cfg_off + WORKING_DIR + "/utils"
color_path_arg = color_off + COLOR_PATH
depth_path_arg = depth_off + DEPTH_PATH
img_num_arg = img_num_off + str(img_num)
output_path_arg = output_off + DATASET_DIR

In [13]:
output_idx = subprocess.check_output(['python3',
                    WORKING_DIR + '/utils' + '/object_detect.py',
                    cfg_path_arg, color_path_arg, depth_path_arg, img_num_arg, output_path_arg])

KeyboardInterrupt: 

In [ ]:
detected_class = []
folder_num = len(os.listdir(DATASET_DIR + '/color_segmented'))
for i in range(folder_num):
    detected_class.append(int(output_idx[len(output_idx)-(2*i+3+i)]+output_idx[len(output_idx)-(2*i+2+i)]))

In [ ]:
# object_num = int(object_num[len(object_num)-2])
# obj_detected_num = len(os.listdir(DATASET_DIR + '/color_segmented'))
obj_detected_num = 0
for i in range(len(os.listdir(DATASET_DIR + '/color_segmented'))):
    temp_path = DATASET_DIR + '/color_segmented/' + valid_class_dict[detected_class[i]]
    for j in range(len(os.listdir(temp_path))):
        obj_detected_num += 1

In [ ]:
# Check obj_detected_num
obj_detected_num

## Get PCD of each objects
#### Issue : Poor pcd result of each object is obtained due to poor total Reconsturction

In [ ]:
# Extract Overlapping points
obj_pcd_num = getPCDAll(img_num, DEPTH_SEG_PATH, RECONST_PCD_PATH, cam_params, detected_class)

In [ ]:
# Check obj_pcd_num
obj_pcd_num

## Apply GPD
#### To get successful result, input point cloud of object should be dense and clear 

In [ ]:
import SharedArray as sa
import time

In [ ]:
from boost_gpd.gpd_interface import grasp_interface_py as gi

In [ ]:
attacth_to_server()

In [ ]:
grasp_list = getGPDAll(obj_pcd_num)

In [ ]:
# Check grasp_list
len(grasp_list)

## Simulate Robot motion

In [3]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7gripper, None,
                INDY_IP)]
              , connection_list=[False])

connection command:
indy0: False


In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [6]:
# geometry 추가
from pkg.geometry.geometry import *
mobile_base = gscene.create_safe(gtype=GEOTYPE.BOX, name="mobile_base", link_name="base_link", 
                   dims=(0.6,0.4,0.439), center=(0,0,-0.439/2), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor = gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link", 
                   dims=(10,10,0.01), center=(0,0,-0.439), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [ ]:
for gtem in gscene:
    round_it_str(gtem.dims)
    round_it_str(gtem.center)

In [ ]:
# Add cam
from pkg.geometry.geotype import GEOTYPE
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam", link_name="indy0_tcp", 
                   dims=(0.061,0.061,0.026), center=(-0.0785,0,0.013), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam_col", link_name="indy0_tcp", 
                   dims=(0.081,0.081,0.046), center=(-0.0785,0,0.013), rpy=(0,0,0), 
                   color=(0.0,1,0,0.3), display=True, fixed=True, collision=True)

viewpoint = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="viewpoint", link_name="indy0_tcp", 
                   dims=(0.01,0.01,0.01), center=(-0.014,0,0), rpy=(0,0,-np.pi/2), 
                   color=(1,0,0,0.3), display=True, fixed=True, collision=False, parent="cam")

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body", link_name="indy0_tcp", 
                   dims=(0.067,0.067,0.0335), center=(-0.0785,0,-0.01675), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, fixed=True, collision=False)

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body_col", link_name="indy0_tcp", 
                   dims=(0.087,0.087,0.0535), center=(-0.0785,0,-0.01675), rpy=(0,0,0), 
                   color=(0.0,1,0,0.3), display=True, fixed=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead", link_name="indy0_tcp", 
                   dims=(0.067,0.067,0.067), center=(-0.0785,0,-0.0335), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, fixed=True, collision=False)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead_col", link_name="indy0_tcp", 
                   dims=(0.087,0.087,0.087), center=(-0.0785,0,-0.0335), rpy=(0,0,0), 
                   color=(0.0,1,0,0.3), display=True, fixed=True, collision=True)

In [ ]:
INIT_POSE = np.deg2rad([  0., -28.,  85.,  -0.,  57., -180])
gscene.show_pose(INIT_POSE)

## Geometry
#### Issue : Ransac fitting result is not invariant. It changes at every trial

In [4]:
import open3d as o3d

In [17]:
pcd = o3d.io.read_point_cloud(WORKING_DIR + '/krylon.pcd')

In [18]:
pcd = pcd.uniform_down_sample(every_k_points=10)

In [19]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
obj_pcd_num = 1

In [13]:
obj_pcd_list = []
for i in range(obj_pcd_num):
    obj_pcd_list.append(o3d.io.read_point_cloud(WORKING_DIR + "/object_{}.pcd".format(i)))

In [5]:
for i in range(obj_pcd_num):
    geometry_matching(np.asarray(o3d.io.read_point_cloud(WORKING_DIR + '/krylon.pcd').points), i)
#     geometry_matching(np.asarray(obj_pcd_list[i].uniform_down_sample(every_k_points=7).points),i)

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [15]:
def add_geometry(T_bo, dims_, geo_type, num):
    if (geo_type == "Cuboid"):
        temp = gscene.create_safe(gtype=GEOTYPE.BOX, name="obj_{}".format(num), link_name="base_link",
            dims=dims_, center=T_bo[:3,3], rpy=Rot2rpy(T_bo[:3,:3]),
            color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
        return temp
        
    elif (geo_type == "Sphere"):
        temp = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="obj_{}".format(num), link_name="base_link",
            dims=dims_, center=T_bo[:3,3], rpy=Rot2rpy(T_bo[:3,:3]),
            color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
        return temp
        
    elif (geo_type == "Cylinder"):
        temp = gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="obj_{}".format(num), link_name="base_link",
            dims=dims_, center=T_bo[:3,3], rpy=Rot2rpy(T_bo[:3,:3]),
            color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
        return temp   

In [16]:
# load해서 geometry 추가

total_geometry = []
total_num = 0
for i in range(obj_pcd_num):
    with open(WORKING_DIR + '/obj_geometry_{}.json'.format(i), 'r') as f:
        json_data = json.load(f)
    geometry_num = len(json_data)
    for j in range(geometry_num):
        total_num += 1
        T_co, dims, geo_type = convertGeometry(json_data, j+1)

        ## Add geometry
        Qcur = np.deg2rad([  0., -28.,  85.,  -0.,  57., -180])
        T_bc = viewpoint.get_tf(list2dict(Qcur, gscene.joint_names))
        T_bo = np.matmul(T_bc, T_co)
        total_geometry.append(add_geometry(T_bo, dims, geo_type, total_num))

## Add Grasp Point 

In [8]:
from pkg.planning.scene import PlanningScene

In [9]:
pscene = PlanningScene(combined_robot=crob, gscene=gscene)

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp",
                   dims=(0.01,)*3, center=(0, 0, 0.14), rpy=(-np.pi/2,0,0), 
                   color=(1,0,0,1), display=True, collision=False, fixed=True)

In [11]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

In [12]:
table = gscene.create_safe(gtype=GEOTYPE.BOX, name="table", link_name="base_link",
                           dims=(0.5,0.5,0.4), center=(0.5, 0, 0.2), rpy=(0,0,0),
                           color=(0.7,0.7,0.7,1), display=True, collision=True, fixed=True)
pscene.create_binder(bname="table", gname="table", _type=PlacePlane, point=None)

In [13]:
goal = gscene.create_safe(gtype=GEOTYPE.BOX, name="goal", link_name="base_link",
                           dims=(0.1,0.1,0.01), center=(0.5, 0.2, 0.395), rpy=(0,0,0),
                           color=(0.7,0.0,0.0,1), display=True, collision=True, fixed=True)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=None)

# Planning pipeline

In [14]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [56]:
cup_cyl = gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cup_cyl", link_name="base_link",
                   dims=(0.05,0.05, 0.07), center=(0.5, 0, 0.436), rpy=(0,0,0),
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
cup_handle = gscene.create_safe(gtype=GEOTYPE.BOX, name="cup_handle", link_name="base_link",
                   dims=(0.02,0.02, 0.05), center=(0.035, 0, 0), rpy=(0,0,0),
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="cup_cyl")

In [69]:
from pkg.planning.constraint.constraint_subject import CustomObject, PlacePoint, DirectedPoint, ConstraintType
##
# @class Grasp2PointFixed
# @brief ActionPoint for rnb-planning.src.pkg.planning.constraint.constraint_actor.Gripper2Tool
class Grasp2PointFixed(DirectedPoint):
    ctype=ConstraintType.Grasp2
    def get_redundancy(self):
        return {}
action_points_dict = {"cup_placement": PlacePoint("cup_placement", cup_cyl, 
                                                        point=[0,0,-cup_cyl.dims[2]/2-0.001], 
                                                        rpy=[0,0,0])}
action_points_dict.update({"cup_grasp": Grasp2PointFixed("cup_grasp", cup_cyl, 
                                                    point=(0,0,0), 
                                                    rpy=Rot2rpy(np.matmul(Rot_rpy((0,-np.pi/2,np.pi/2)), Rot_axis(3, -np.pi/4))))})
obj_pscene = pscene.create_subject(oname="cup", gname="cup_cyl", _type=CustomObject, 
                             action_points_dict=action_points_dict)

In [70]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner

In [71]:
mplan = MoveitPlanner(pscene)
ppline.set_motion_planner(mplan=mplan)

In [72]:
from pkg.planning.task.rrt import TaskRRT

In [73]:
tplan = TaskRRT(pscene)
ppline.set_task_planner(tplan)

In [74]:
initial_state = pscene.initialize_state(crob.home_pose)

In [75]:
goal_nodes = [('goal',)]

In [76]:
ppline.search(initial_state=initial_state, goal_nodes=goal_nodes, 
              verbose=True, add_homing=True, 
              timeout=1, timeout_loop=10)

try: 0 - ('table',)->('grip0',)
try transition motion
transition motion tried: True
result: 0 - ('table',)->('grip0',) = success
branching: 0->1 (0.08/10.0 s, steps/err: 55(81.4809799194 ms)/0.00165204277128)
try: 1 - ('grip0',)->('goal',)
try transition motion
transition motion tried: True
result: 1 - ('grip0',)->('goal',) = success
branching: 1->2 (0.14/10.0 s, steps/err: 60(59.2229366302 ms)/0.00157030975649)
++ adding return motion to acquired answer ++
======================= terminated 0: first answer acquired ===============================


In [77]:
pscene.set_object_state(initial_state)

In [78]:
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

In [79]:
ppline.play_schedule(snode_schedule=snode_schedule, period=0.01)

('table',)->('table',)
('table',)->('grip0',)
('grip0',)->('goal',)
('goal',)->('goal',)
